**Setup**

In [1]:
!pip install spotipy

In [23]:
import sys
import json
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd

**Authentication**

In [3]:
username = "tlarson07"
scope = "user-library-read"

client_credentials_manager = SpotifyClientCredentials(client_id='c2cf260ff28e4694bcb1eb0a29191b74', client_secret='83713ae562c04d7e96220f60749a6989')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

**Playground**

In [5]:
results = sp.search("Mylo Xyloto",limit=3,type="album")['albums']['items']
print(len(results))

3


In [6]:
mx_tracks = sp.album_tracks(album_id=results[0]['id'])['items']

In [7]:
mx_tracks[0]['uri']

'spotify:track:3vCzHYgSjMuGjFMfJSCx4c'

In [8]:
mx_tracks[1]['uri']

'spotify:track:6WF4hzdGXvXd1joERSXJjm'

In [9]:
results = sp.search("Golden Slumbers",limit=3,type="track")['tracks']['items']
print(len(results))

3


In [10]:
results[0]['uri']

'spotify:track:01SfTM5nfCou5gQL70r6gs'

In [11]:
trackname = 'Hyper Music'
results = sp.search(trackname,limit=3,type="track")['tracks']['items']
print(results[0]['uri'], results[0]['name'], results[0]['album']['name'], results[0]['artists'][0]['name'],sep='\n')

spotify:track:3LwfE3USKwANPjPz34iGDZ
Hyper Music
Origin Of Symmetry
Muse


In [12]:
trackname = 'Plug in Baby'
results = sp.search(trackname,limit=3,type="track")['tracks']['items']
print(results[0]['uri'], results[0]['name'], results[0]['album']['name'], results[0]['artists'][0]['name'],sep='\n')

spotify:track:429MqIq5nGH32feyRZHunv
Plug In Baby
Origin Of Symmetry
Muse


In [13]:
np.array(list(map(lambda x : x**2,[1,2,3,4,5])))

array([ 1,  4,  9, 16, 25])

**Sanitizing Data**

In [14]:
def getMetaData(songID):
    """
    Assume spotipy has already been initiated as sp
    songID is a string that indicates the spotify ID
    """
    results = sp.audio_analysis(songID)
    last_section = results['sections'][-1]
    last_section_time = last_section['duration']
    segment_durations = 0
    segment_list = []
    for segment in results['segments'][::-1]:
        segment_list.append(segment)
        segment_durations = segment_durations + segment['duration']
        if segment_durations > last_section_time:
            break
            
    ## Segment Analysis
    all_pitches = np.array(list(map(lambda x : np.array(x['pitches'])*x['duration']/segment_durations, segment_list)))
    all_pitches = np.mean(all_pitches,axis=0)
    all_timbre = np.array(list(map(lambda x : np.array(x['timbre'])*x['duration']/segment_durations, segment_list)))
    all_timbre = np.mean(all_timbre,axis=0)
    
    
    return np.concatenate([all_pitches,all_timbre,
                           [last_section['key']],[last_section['mode']],
                           [last_section['tempo']],[last_section['time_signature']]])

In [15]:
column_names = ["pitch_{}".format(i) for i in range(12)] + ["timbre_{}".format(i) for i in range(12)] + ['key','mode','tempo','time_signature']

In [16]:
getMetaData("spotify:track:429MqIq5nGH32feyRZHunv")

array([ 4.52818603e-03,  5.34589441e-03,  2.26670455e-02,  4.87355205e-03,
        6.53165212e-03,  4.14161658e-03,  5.01310395e-03,  2.70174234e-02,
        4.69997068e-03,  9.63814002e-03,  3.03260382e-03,  6.70987349e-03,
        1.28452361e+00,  1.57542692e+00, -3.89197561e-01,  2.59472834e+00,
       -6.36650510e-01, -1.07673244e+00, -9.17548014e-01,  2.14649444e-01,
       -1.85827140e-01, -5.98507793e-02, -8.38777259e-01,  2.51917588e-01,
        2.00000000e+00,  1.00000000e+00,  1.34974000e+02,  4.00000000e+00])

In [17]:
sp.track('spotify:track:429MqIq5nGH32feyRZHunv').keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [18]:
results = sp.audio_analysis("spotify:track:429MqIq5nGH32feyRZHunv")

In [19]:
results.keys()

dict_keys(['meta', 'track', 'bars', 'beats', 'tatums', 'sections', 'segments'])

In [20]:
results['sections'][-1]

{'start': 204.40184,
 'duration': 13.59816,
 'confidence': 1.0,
 'loudness': -14.265,
 'tempo': 134.974,
 'tempo_confidence': 0.232,
 'key': 2,
 'key_confidence': 0.033,
 'mode': 1,
 'mode_confidence': 0.496,
 'time_signature': 4,
 'time_signature_confidence': 1.0}

In [21]:
results['segments'][-1]

{'start': 214.49728,
 'duration': 3.50272,
 'confidence': 1.0,
 'loudness_start': -28.588,
 'loudness_max_time': 0.01422,
 'loudness_max': -14.033,
 'loudness_end': -60.0,
 'pitches': [0.048,
  0.053,
  0.941,
  0.03,
  0.03,
  0.046,
  0.024,
  1.0,
  0.028,
  0.036,
  0.021,
  0.072],
 'timbre': [29.809,
  49.862,
  -118.811,
  300.273,
  -14.522,
  -63.36,
  -71.816,
  53.15,
  -9.461,
  -10.152,
  -83.869,
  9.945]}

In [25]:
df = pd.read_csv('songs.csv')

In [ ]:
tags = pd.read_csv("tags.csv")

In [ ]:
tags = tags.drop(tags.columns[0],axis=1)

In [29]:
firstCols = ['first_pitch_0', 'first_pitch_1', 'first_pitch_2', 'first_pitch_3',
       'first_pitch_4', 'first_pitch_5', 'first_pitch_6', 'first_pitch_7',
       'first_pitch_8', 'first_pitch_9', 'first_pitch_10', 'first_pitch_11',
       'first_timbre_0', 'first_timbre_1', 'first_timbre_2', 'first_timbre_3',
       'first_timbre_4', 'first_timbre_5', 'first_timbre_6', 'first_timbre_7',
       'first_timbre_8', 'first_timbre_9', 'first_timbre_10',
       'first_timbre_11','first_tempo','acousticness_1', 'danceability_1', 'energy_1',
       'instrumentalness_1', 'liveness_1', 'loudness_1',
       'speechiness_1']

In [30]:
secondCols = ['second_pitch_0', 'second_pitch_1', 'second_pitch_2','second_pitch_3',
 'second_pitch_4','second_pitch_5','second_pitch_6','second_pitch_7', 'second_pitch_8',
 'second_pitch_9','second_pitch_10', 'second_pitch_11','second_timbre_0','second_timbre_1',
 'second_timbre_2','second_timbre_3','second_timbre_4','second_timbre_5', 'second_timbre_6',
 'second_timbre_7','second_timbre_8', 'second_timbre_9','second_timbre_10',
 'second_timbre_11', 'second_tempo', 'acousticness_2', 'danceability_2','energy_2',
 'instrumentalness_2','liveness_2','loudness_2','speechiness_2']

In [32]:
params = []
for i,j in zip(firstCols,secondCols):
    fulldf['diff_{}_{}'.format(i,j)] = fulldf[i]-fulldf[j]
    params.append("diff_{}_{}".format(i,j))

NameError: name 'fulldf' is not defined

**final data set**

In [ ]:
fulldf

,Rating,first_track,first_artist,first_album,first_uri,first_pitch_0,first_pitch_1,first_pitch_2,first_pitch_3,first_pitch_4,...,diff_first_timbre_10_second_timbre_10,diff_first_timbre_11_second_timbre_11,diff_first_tempo_second_tempo,diff_acousticness_1_acousticness_2,diff_danceability_1_danceability_2,diff_energy_1_energy_2,diff_instrumentalness_1_instrumentalness_2,diff_liveness_1_liveness_2,diff_loudness_1_loudness_2,diff_speechiness_1_speechiness_2
0,0.0,Palm Dreams,Hayley Kiyoko,Expectations,spotify:track:3qDaH8tgcdzdpxaL7CyW2s,0.003565,0.004857,0.004563,0.002404,0.003796,...,0.264841,-0.059673,18.989,0.112660,0.441,0.3130,-0.022237,-0.1910,4.704,0.0141
1,0.0,Let It Be,Hayley Kiyoko,Expectations,spotify:track:181uk7o0ajEG8F1HYrqbxu,0.009317,0.006336,0.013441,0.004119,0.010720,...,1.910682,0.151112,-58.402,-0.318000,0.172,0.7050,0.000004,0.0352,8.806,-0.0564
2,0.0,TOOTIMETOOTIMETOOTIME,The 1976,A Brief Inquiry Into Online Relationships,spotify:track:33DzKnwuBE6lfOiADwzd5E,0.005280,0.003165,0.001918,0.006115,0.001714,...,0.472716,0.510871,-20.303,-0.569000,0.254,0.0060,-0.277000,0.0245,2.016,-0.1935
3,0.0,Vibe,Tove Stryke,NaN,spotify:track:7rXOrI115JVyhBfM2DRKwZ,0.023489,0.034048,0.018352,0.050424,0.031022,...,-0.665551,3.493595,-173.393,-0.414000,0.081,0.3210,0.000000,-0.3100,3.426,-0.1048
4,0.0,Good Mourning,Halsey,hopeless fountain kingdom,spotify:track:6Ck8Lq4srTWQP1PXZ9P8aZ,0.071429,0.038957,0.013337,0.017681,0.012524,...,-0.164249,0.161687,-101.743,0.490000,-0.002,-0.1750,0.027368,0.2840,-7.002,0.0212
5,0.0,Behind These Hazel Eyes,Kelly Clarkson,Breakaway,spotify:track:0AKAxdNkwq9ZxRdW1DN9zW,0.016384,0.037209,0.044753,0.024586,0.040129,...,10.535084,2.434488,-70.080,-0.028150,0.102,0.1880,0.001081,0.1039,-0.503,0.0087
6,0.0,Lost Myself,Apollos,NaN,spotify:track:0wBUHmaeihAuTWPbN3Y6Rc,0.010065,0.002190,0.001884,0.004832,0.002197,...,-0.194212,0.297531,-4.459,0.164540,-0.172,-0.1310,0.000002,-0.0630,-2.410,-0.0045
7,0.0,California Gurls,Katy Perry,Teenage Dream,spotify:track:6tS3XVuOyu10897O3ae7bi,0.003834,0.003294,0.001873,0.001280,0.001187,...,0.435384,1.796524,124.992,-0.738540,0.110,0.4620,0.000000,0.0656,5.870,0.0086
8,0.0,Never Not,Lauv,I met you when I was 18.,spotify:track:4fFfyouye6vrX9o9z4PVMu,0.033520,0.019650,0.018931,0.018360,0.021760,...,-0.605904,-0.854682,27.399,0.676200,-0.048,-0.3660,-0.000001,-0.1020,-1.306,-0.0829
9,0.0,How It Ends,Emily Warren,NaN,spotify:track:06uX6D1YUwbzBhpRBpXL5L,0.005292,0.003806,0.004184,0.007791,0.059998,...,-1.037733,-0.732631,132.546,-0.747400,-0.229,0.4130,-0.003040,-0.0194,7.446,0.1020


**Models**

_1. LogisticRegression_

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
np.array(fulldf['Rating'])

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,
        1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,
        1. ,  1. ,  1. ])

In [ ]:
scoretest = fulldf['Rating'].map(lambda x : 1 if x>0 else x)
scoretest.value_counts()/len(scoretest)

1.0    0.636986
0.0    0.363014
Name: Rating, dtype: float64

In [ ]:
model.fit(y=fulldf['Rating'].map(lambda x : 1 if x>0 else x),X=fulldf[params])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
scores = model.predict(fulldf[params])-fulldf['Rating']

In [ ]:
scores = scores **2
scores = scores.mean()
scores**.5

0.6620847108818944

_2. LinearSVC_

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
X = fulldf[params]
Y = fulldf['Rating'].map({1:'Bleed',0.5:'Flow',0:'NC'})

In [ ]:
>>> lin_clf = LinearSVC()
>>> lin_clf.fit(X, Y) 

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [ ]:
np.array(lin_clf.predict(X) == fulldf['Rating'].map({1:'Bleed',0.5:'Flow',0:'NC'})).mean()

0.5547945205479452

In [18]:
# prediction counts
pd.Series(lin_clf.predict(X)).value_counts()

NameError: name 'pd' is not defined

In [19]:
# actual counts
fulldf['Rating'].map({1:'Bleed',0.5:'Flow',0:'NC'}).value_counts()

NameError: name 'fulldf' is not defined

_3. SVC_

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf = SVC()
clf.fit(X, Y) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
clf.predict(X)

array(['NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'Flow', 'NC', 'NC', 'NC', 'NC',
       'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC',
       'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'Flow', 'NC',
       'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC',
       'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'Flow',
       'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow',
       'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow',
       'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow',
       'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow',
       'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow',
       'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow',
       'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow',
       'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow', 'Flow',
       'Flow', 'Flow', 'Flow', 'Bleed', 'Bleed', 'Bleed', 'Bleed', 

In [ ]:
np.array(clf.predict(X) == Y).mean()

0.95205479452054798

In [ ]:
mask = np.random.rand(146)>.2

In [ ]:
clf.fit(X[mask], Y[mask]) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
np.array(clf.predict(X[~mask]) == Y[~mask]).mean()

0.47999999999999998